In [100]:
import optuna
import numpy as np
import pandas as pd
from dvclive import Live
from optuna.trial import Trial
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [101]:
random_state = 42
np.random.seed(random_state)

### Dataset

In [102]:
df = pd.read_csv("data/Ethos_Dataset_Binary.csv", sep=";")
df

,comment,isHate
0,You should know women's sports are a joke,1.0
1,You look like Sloth with deeper Down’s syndrome,1.0
2,You look like Russian and speak like Indian. B...,1.0
3,"Women deserve to be abused, I guess.",1.0
4,Women are made for making babies and cooking d...,1.0
...,...,...
993,From the midnight sun where the hot springs blow,0.0
994,Don't say I'm not your type,0.0
995,And therefore never send to know for whom the...,0.0
996,And I can't stand another day,0.0


In [103]:
df["isHate"].unique()

array([1.        , 0.98387097, 0.98360656, 0.97826087, 0.97333333,
       0.96666667, 0.95454545, 0.94545455, 0.9375    , 0.90384615,
       0.85714286, 0.8490566 , 0.84615385, 0.83333333, 0.82142857,
       0.75      , 0.72222222, 0.67857143, 0.66666667, 0.60344828,
       0.53061224, 0.5       , 0.4       , 0.33333333, 0.30232558,
       0.296875  , 0.25      , 0.2       , 0.16666667, 0.16071429,
       0.15254237, 0.11111111, 0.10344828, 0.09090909, 0.03896104,
       0.03773585, 0.03174603, 0.03030303, 0.02985075, 0.02631579,
       0.01886792, 0.01639344, 0.        ])

Очень хорошо видно, что, хоть датасет и позиционирует себя как датасет бинарной классификации хейтспича, значения `isHate` в нем небинарные :)<br>
Возьмем трешхолд хейтспича = 0.5 и округлим значения больше трешхолда до 1, меньше - до 0, значения, которые равны трешхолду, случайным образом перемешаем между классами. 

In [104]:
threshold = 0.5
t_size = len(df[df['isHate'] == threshold])
df.loc[df['isHate'] > threshold, "isHate"] = 1
df.loc[df['isHate'] < threshold, "isHate"] = 0
df.loc[df['isHate'] == threshold, "isHate"] = np.random.randint(0, 2, size=(t_size))
df["isHate"].unique()

array([1., 0.])

Разобьем датасет на тренировочную и тестовую выборки с помощью `train_test_split`

In [105]:
train_size = 0.8
df_train, df_test = train_test_split(
    df, train_size=train_size, random_state=random_state
)
len(df_train), len(df_test)

(798, 200)

### Model Selection

Будем использовать `random forest`<br>
В качестве фичей возьмем эмбеддинги текста на основе `tf-idf` векторизации

Проверим, что модель обучается с фиксированным набором гиперпараметров

In [106]:
corpora_train = df_train["comment"].to_list()
corpora_test = df_test["comment"].to_list()
y_train = df_train["isHate"].to_numpy()
y_test = df_test["isHate"].to_numpy()

In [107]:
vectorizer = TfidfVectorizer(max_features=300)
emb_train = vectorizer.fit_transform(corpora_train).toarray()
emb_test = vectorizer.transform(corpora_test).toarray()

In [108]:
forest = RandomForestClassifier()
forest.fit(emb_train, y_train)
forest.score(emb_test, y_test)

0.67

### Hyperparameter Optimization & DVC Experiment Tracking

Для подбора гиперпараметров будем использовать библиотеку `optuna`<br>
Логгировать результаты экспериментов будем с помощью библиотеки `DVCLive`

К сожалению, интеграция `DVCLive` c `optuna` [неполная](https://github.com/iterative/dvclive/issues/118#issuecomment-1285863519), и по умолчанию `DVCLive` сохраняет каждый отдельный запуск триала как отдельный эксперимент<br>
Это не подходит под наши цели, так как всё логгирование осуществяется внутри гита и для сравнения метрик нам бы пришлось каждый раз менять различные коммиты экспериментов через `.git/refs`

Изменим это поведение, обернув `objective` функцию внутри `Live` блока, таким образом мы будем сохранять значения гиперпараметров вместе с метриками на каждой итерации их поиска<br>

Логгировать будем следующие сущности:
- `n_embs_dim` - размерность эмбеддингов `tf-idf`
- `n_estimators` - количество деревьев в лесу
- `accuracy` - точность обученного классификатора на тестовой выборке

Метрики (в нашем случае и гиперараметры) на каждой итерации триала логгируются в свой `.tsv` файл, расположенный в каталоге `experiments/dvc-optuna/plots/metrics`<br>

Дополнительно сохраним 'статичные' гиперпараметры, такие как:
- `seed` - начальное состояние ГПСЧ
- `train_size` - размер тренировной выборки

Данные значения логгируются в файл `experiments/dvc-optuna/params.yaml`

In [109]:
with Live("experiments/dvc-optuna", save_dvc_exp=True) as live:
    def objective(trial: Trial):
        n_embs_dim = trial.suggest_int("n_embs_dim", 100, 768)
        vectorizer = TfidfVectorizer(max_features=n_embs_dim)
        emb_train = vectorizer.fit_transform(corpora_train).toarray()
        emb_test = vectorizer.transform(corpora_test).toarray()
        n_estimators = trial.suggest_int("n_estimators", 1, 500)
        criterion = trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"])
        forest = RandomForestClassifier(
            criterion=criterion, n_estimators=n_estimators, random_state=random_state
        )
        forest.fit(emb_train, y_train)
        score = forest.score(emb_test, y_test)
        live.log_metric("n_embs_dim", n_embs_dim)
        live.log_metric("n_estimators", n_estimators)
        live.log_metric("accuracy", score)
        live.next_step()
        return score
    live.log_param("seed", random_state)
    live.log_param("train_size", train_size)
    sampler = optuna.samplers.TPESampler(seed=random_state)
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(objective, n_trials=50)
best_params, best_value = study.best_params, study.best_value
print(best_params, best_value)

[I 2023-06-11 16:17:41,941] A new study created in memory with name: no-name-b83d68f2-44e1-4d40-a1b3-b45edf776c6a
[I 2023-06-11 16:17:43,320] Trial 0 finished with value: 0.695 and parameters: {'n_embs_dim': 350, 'n_estimators': 476, 'criterion': 'gini'}. Best is trial 0 with value: 0.695.
[I 2023-06-11 16:17:43,425] Trial 1 finished with value: 0.65 and parameters: {'n_embs_dim': 204, 'n_estimators': 30, 'criterion': 'gini'}. Best is trial 0 with value: 0.695.
[I 2023-06-11 16:17:44,732] Trial 2 finished with value: 0.66 and parameters: {'n_embs_dim': 113, 'n_estimators': 485, 'criterion': 'gini'}. Best is trial 0 with value: 0.695.
[I 2023-06-11 16:17:45,192] Trial 3 finished with value: 0.65 and parameters: {'n_embs_dim': 222, 'n_estimators': 153, 'criterion': 'gini'}. Best is trial 0 with value: 0.695.
[I 2023-06-11 16:17:45,507] Trial 4 finished with value: 0.69 and parameters: {'n_embs_dim': 509, 'n_estimators': 70, 'criterion': 'log_loss'}. Best is trial 0 with value: 0.695.
[I 

{'n_embs_dim': 389, 'n_estimators': 497, 'criterion': 'gini'} 0.715


### Model evaluation

Снова обучим модель, используя оптимальные гиперпараметры

In [110]:
vectorizer = TfidfVectorizer(max_features=best_params["n_embs_dim"])
emb_train = vectorizer.fit_transform(corpora_train).toarray()
emb_test = vectorizer.transform(corpora_test).toarray()
forest = RandomForestClassifier(
    criterion=best_params["criterion"],
    n_estimators=best_params["n_estimators"],
    random_state=random_state
)
forest.fit(emb_train, y_train)
forest.score(emb_test, y_test)

0.715

Посмотрим на значения метрик для тестовый выборки

In [111]:
y_test_pred = forest.predict(emb_test)
pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True))

,0.0,1.0,accuracy,macro avg,weighted avg
precision,0.741007,0.655738,0.715,0.698372,0.708605
recall,0.830645,0.526316,0.715,0.678480,0.715000
f1-score,0.783270,0.583942,0.715,0.683606,0.707525
support,124.000000,76.000000,0.715,200.000000,200.000000


и для тренировочной

In [112]:
y_train_pred = forest.predict(emb_train)
pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))

,0.0,1.0,accuracy,macro avg,weighted avg
precision,0.995825,1.000000,0.997494,0.997912,0.997504
recall,1.000000,0.993769,0.997494,0.996885,0.997494
f1-score,0.997908,0.996875,0.997494,0.997391,0.997492
support,477.000000,321.000000,0.997494,798.000000,798.000000


Очень хорошо видно, что случайный лес переобучился на тренировочных данных